<a href="https://colab.research.google.com/github/joanneim/code_samples/blob/main/api_pull_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About the Google Knowedge Graph API

*   Payment/API query quota: The Knowledge Graph API Search API allows developers a free quota of up to 100,000 (one hundred thousand) read calls per day per project

*   A useful primer can be found here: https://searchengineland.com/
laymans-visual-guide-googles-knowledge-graph-search-api-241935

*   The test GUI can be found here:
https://developers.google.com/knowledge-graph/reference/rest/v1/?apix=true





# Code

In [ ]:
!pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Dependencies
%%capture
from __future__ import print_function
import json
import urllib
import pandas as pd
import time
from google.colab import drive
import urllib.request


In [ ]:
# DO NOT SHARE. IF YOU SEE THIS AND YOU ARE NOT ME KINDLY NOTIFY AUTHOR IMMEDIATELY.
api_key = API_KEY

In [ ]:
INPUT_FILE_NAME  = "notickers_sellers.xlsx"
OUTPUT_FILE_NAME = "notickers_sellers_out.xlsx"

In [ ]:
# Mount Google Drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Specify entity type, which is required in query to Google's Knowledge Graph
# param_entitytype in ["Corporation", "Entity", "Organization", "Thing", "Place"]

param_entitytype = "Entity"

In [ ]:
# Import excel spradsheet of parent company names that was generated from STATA code
df = pd.read_excel("/content/drive/My Drive/OwnershipPatterns/data_in/" + INPUT_FILE_NAME)
print(df.columns)

Index(['sellername', 'sellername_first', 'sellername_second'], dtype='object')


In [ ]:
# Extract 'Key people' field from company page infobar;
google_v =  ['google_name', 'google_score', 'google_url']
global fields
fields = ['Formerly', 'Defunct', 'Fate' , 'Successor', 'Parent', 'Type', 'Traded as', 'Industry', 'ISIN', 'Headquarters', 'Area served', 'Total assets', 'Net income', 'Operating income', 'Total equity', 'Number of employees']
for f in google_v + fields:
  df.insert(loc = 0, column = f, value = ['' for i in range(df.shape[0])])



In [ ]:
# Example query
query = "Linde Plc"
param_entitytype = "Corporation"
service_url = 'https://kgsearch.googleapis.com/v1/entities:search'
params = {
    'query': query,
    'limit': 1,
    'indent': True,
    'key': api_key,
    'types': param_entitytype
}
url = service_url + '?' + urllib.parse.urlencode(params)
response = json.loads(urllib.request.urlopen(url).read())
print(response)


{'@context': {'kg': 'http://g.co/kg', 'goog': 'http://schema.googleapis.com/', 'EntitySearchResult': 'goog:EntitySearchResult', 'detailedDescription': 'goog:detailedDescription', 'resultScore': 'goog:resultScore', '@vocab': 'http://schema.org/'}, '@type': 'ItemList', 'itemListElement': [{'@type': 'EntitySearchResult', 'result': {'@type': ['Organization', 'Thing', 'Corporation'], 'description': 'Chemicals company', 'image': {'url': 'https://commons.wikimedia.org/wiki/File:Linde_plc.jpg', 'contentUrl': 'https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcRjoogaxC8b6zC0G4wF1TCVtErzDLIdX1b65nW0u_qev5czRH1P'}, '@id': 'kg:/g/1234zd1f', 'name': 'Linde plc', 'detailedDescription': {'url': 'https://en.wikipedia.org/wiki/Linde_plc', 'license': 'https://en.wikipedia.org/wiki/Wikipedia:Text_of_Creative_Commons_Attribution-ShareAlike_3.0_Unported_License', 'articleBody': "Linde plc is a global multinational chemical company founded in Germany and, since 2018, domiciled in Ireland and headquartere

In [ ]:
def _empty(x):

  ''' evaluates whether input is empty '''

  try:
    status = (x==None)
  except:
    pass

  if status==False:
    try:
      status = (pd.isnull(x))
    except:
      pass

  if status==False:
    try:
      status = len(x)==0
    except:
      pass

  if status==False:
    try:
      status = x==""
    except:
      pass

  if status==False:
    try:
      status = (pd.isna(x))
    except:
      pass

  if status==False:
    try:
      status = (x=="nan")
    except:
      pass

  if status==False:
    return False
  else:
    return True


In [ ]:
def gk_wiki_data_extractor(i, df_field = "stn_name", df_type = "Entity"):
  '''
  Preamble: This function systematicaly extracts firm level data from Google's Knowlege Graph
  Inputs:
    - row i of dataframe with firm names
    - df_field: dataframe field containing firm name
    - df_type: param_entitytype, which can be one of the following ["Corporation", "Entity", "Organization", "Thing", "Place"]
  '''

  query = df.loc[i, df_field]
  param_entitytype = df_type
  service_url = 'https://kgsearch.googleapis.com/v1/entities:search'
  params = {
      'query': query,
      'limit': 1,
      'indent': True,
      'key': api_key,
      'types': param_entitytype
  }


  url = service_url + '?' + urllib.parse.urlencode(params)
  try:
    response = json.loads(urllib.request.urlopen(url).read())
  except:
    return

  if len(response['itemListElement'])>0:

    for item in response['itemListElement'][0]['result']:

      name = response['itemListElement'][0]['result']['name']
      score = response['itemListElement'][0]['resultScore']

      try:
        url = response['itemListElement'][0]['result']['detailedDescription']['url']

        df.loc[i, 'google_name'], df.loc[i, 'google_score'], df.loc[i, 'google_url'] = name, score, url


        ## get wiki data ##

        infobar = pd.read_html(url)
        df_wiki = infobar[0]

        if len(df_wiki.columns) == 2:
            df_wiki.columns = ['columns', 'data']
            # most output tables have 2 elements
        else:
            df_wiki.columns = ['columns', 'data', 'trash']
            df_wiki.drop(columns = 'trash', inplace = True)
            # sometimes the table output has 3 elements

        # some table cleaning is needed before transposing
        df_wiki.set_index(df_wiki['columns'], inplace = True)
        df_wiki.drop(columns = 'columns', inplace = True)
        df_wiki = df_wiki.transpose()
        # fields
        for f in fields:
          if f in df_wiki.columns:
            df.loc[i, f] = df_wiki[f][0]

        return
      except:
        pass

In [ ]:
def wiki_data_extractor(i, df_field):
  '''
  Preamble: This function systematicaly extracts firm level data from Wikipedia
  Inputs:
  - row i of dataframe with firm names
  - df_field: dataframe field containing firm name
  '''
    try:

      # Must properly capitalize the name
      wiki_url = "https://en.wikipedia.org/wiki/" + df.loc[i, df_field].replace(" ", "_")

      df.loc[i, 'google_name'] , df.loc[i, 'google_url'] = df.loc[i, df_field] ,  wiki_url
      infobar = pd.read_html(wiki_url)

      for k in range(2):

        if _empty(df.loc[i,'Traded as'])==True:
          # Sometimes the infobar is infobar[0]; sometimes, infobar[2]
          df_wiki = infobar[k]
          if len(df_wiki.columns) == 2:
              df_wiki.columns = ['columns', 'data']
              # most output tables have 2 elements
          else:
              df_wiki.columns = ['columns', 'data', 'trash']
              df_wiki.drop(columns = 'trash', inplace = True)
              # sometimes the table output has 3 elements

          # some table cleaning is needed before transposing
          df_wiki.set_index(df_wiki['columns'], inplace = True)
          df_wiki.drop(columns = 'columns', inplace = True)
          df_wiki = df_wiki.transpose()
          # fields
          for f in fields:
            if f in df_wiki.columns:
              df.loc[i, f] = df_wiki[f][0]

    except:
        pass

In [ ]:
"""Python client calling Knowledge Graph Search API."""

"""
    Preamble: The excel spreadsheet records around 4, 759 unique entities.
    For each entry, this script submits a Knowledge Graph query and extracts
    the first result that is an Organization.

    Code runs for approximately five minutes.

"""

for i in range(0, df.shape[0]):
#for i in range(1791, 2338):
#for i in range(1791, 1800):
#for i in range(1648, 1652):
  #for i in range(1500, df.shape[0]):

  # Iteration number
  print(i)

  # Tri wiki directly
  if _empty(df.loc[i, 'Traded as'])==True:
    wiki_data_extractor(i, "sellername")


    if _empty(df.loc[i, 'Traded as'])==True:
      wiki_data_extractor(i, "sellername")


      # May not be traded. once we do a stn_name_s we run the risk of
      # landing on a wiki page for a very generic item
      if _empty(df.loc[i, 'Traded as'])==True & _empty(df.loc[i, 'Headquarters'])==True:
        wiki_data_extractor(i, "sellername_second")

        if _empty(df.loc[i, 'Traded as'])==True & _empty(df.loc[i, 'Headquarters'])==True:
          wiki_data_extractor(i, "sellername")

        if _empty(df.loc[i, 'Traded as'])==True & _empty(df.loc[i, 'Headquarters'])==True:
          wiki_data_extractor(i, "sellername_first")


  if _empty(df.loc[i, 'Traded as'])==True & _empty(df.loc[i, 'Headquarters'])==True:
    gk_wiki_data_extractor(i, "sellername")

  if _empty(df.loc[i, 'Traded as'])==True & _empty(df.loc[i, 'Headquarters'])==True:
    gk_wiki_data_extractor(i, "sellername_second")


  # Update
  print(df.iloc[i])
  if (i!=0) & (i%10==0):
    print(df.iloc[i])
  # Introduce a pause every 100th iterations to circumvent HTTP error 429: Too Many Requests
  # Save also as a pickle
  if (i!=0) & (i%100==0):
    time.sleep(60)
    df.to_pickle("/content/drive/My Drive/OwnershipPatterns/data_out/" + OUTPUT_FILE_NAME)

  # Submit Knowledge Graph query

# Final pickle
df.to_pickle("/content/drive/My Drive/OwnershipPatterns/data_out/" + OUTPUT_FILE_NAME)

Number of employees                                           
Total equity                                                  
Operating income                                              
Net income                                                    
Total assets                                                  
Area served                                                   
Headquarters                                                  
ISIN                                                          
Traded as                                                     
Type                                                          
Parent                                                        
Successor                                                     
Fate                                                          
Defunct                                                       
Formerly                                                      
google_url             https://en.wikipedia.org/wiki/pu

In [ ]:
df.to_excel("/content/drive/My Drive/OwnershipPatterns/data_out/" + OUTPUT_FILE_NAME + ".xlsx",
             sheet_name='Sheet_name_1')